## MNIST dataset with Tensorflow: 

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist


In [4]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
print(xtrain.shape, ytrain.shape)

(60000, 28, 28) (60000,)


In [5]:
# We need to flatten xtrain in order to set as input for neural network nd it will be numpy array :
# We also need to convert float64 to float32 to minimize computations :
# will devide by 255 coz it is grayscale and we need to normalize :
xtrain = xtrain.reshape(-1, 28*28).astype("float32") / 255.0
xtest = xtest.reshape(-1, 28*28).astype("float32") / 255.0
print(xtrain.shape, xtest.shape)

# This can also done by tensorflow as 
# xtrain = tf.convert_to_tensor(xtrain)
# Similarly for xtest----


(60000, 784) (10000, 784)


In [4]:
# Sequential API (very convenient but not very flexible xoz it map 1 input to 1 output)
model = keras.Sequential(
    [
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10),
    ]
)
# IF we do not use Softmax, we can use from_logits=True in loss function that will help to send softmax first :
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # IF ur Y is hot encoded then use just categorical_crossentropy and if Y is integer USE sparse_categorical_crossentropy
    optimizer= keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"],
)

model.fit(xtrain, ytrain, batch_size=100, epochs=5, verbose=2)
model.evaluate(xtest, ytest, batch_size=32, verbose=2)

# By adding input above dense as keras.input(shape=(28*28)) , we can get summary model.summary()

Train on 60000 samples
Epoch 1/5
60000/60000 - 8s - loss: 0.2129 - accuracy: 0.9382
Epoch 2/5
60000/60000 - 3s - loss: 0.0789 - accuracy: 0.9751
Epoch 3/5
60000/60000 - 3s - loss: 0.0497 - accuracy: 0.9840
Epoch 4/5
60000/60000 - 4s - loss: 0.0368 - accuracy: 0.9881
Epoch 5/5
60000/60000 - 4s - loss: 0.0297 - accuracy: 0.9901
10000/10000 - 1s - loss: 0.0772 - accuracy: 0.9784


[0.07722423628928372, 0.9784]

In [5]:
# Functional API (bit more flexible)
inputs = keras.Input(shape=(784))
x = layers.Dense(512, activation='relu')(inputs)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(10, activation='sigmoid')(x)
Model = keras.Model(inputs=inputs, outputs=outputs)

Model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer= keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"],
)

Model.fit(xtrain, ytrain, batch_size=100, epochs=5, verbose=2)
Model.evaluate(xtest, ytest, batch_size=32, verbose=2)



Train on 60000 samples
Epoch 1/5
60000/60000 - 4s - loss: 0.2388 - accuracy: 0.9319
Epoch 2/5
60000/60000 - 4s - loss: 0.0889 - accuracy: 0.9724
Epoch 3/5
60000/60000 - 5s - loss: 0.0574 - accuracy: 0.9821
Epoch 4/5
60000/60000 - 6s - loss: 0.0418 - accuracy: 0.9869
Epoch 5/5
60000/60000 - 5s - loss: 0.0297 - accuracy: 0.9906
10000/10000 - 1s - loss: 0.0697 - accuracy: 0.9810


[0.06970541148988996, 0.981]

## CIFAR10 Dataset with Tensorflow:

In [11]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [12]:
# Like before we do not rehape to flatten layer coz we will use convolution model so we will maintain orignal shape height, wid:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
print(x_train.shape, x_test.shape)


(50000, 32, 32, 3) (10000, 32, 32, 3)


In [13]:
# Now we will build model : 
modell = keras.Sequential()
modell.add(keras.Input(shape=(32,32,3)))
# In layer convnet we will use filter size (3,3) we can write just 3 and also padding VALID OR SAME, default valid.
# SAME, they are going to maintain shape, if use VALID, they will change shape WRT kernel size.
modell.add(layers.Conv2D(32, 3, padding='valid', activation='relu'))
modell.add(layers.MaxPooling2D(pool_size=(2,2)))
modell.add(layers.Conv2D(64, 3, activation='relu'))
modell.add(layers.MaxPooling2D())
modell.add(layers.Conv2D(128, 3, activation='relu'))
modell.add(layers.Flatten())
modell.add(layers.Dense(64, activation='relu'))
modell.add(layers.Dense(10))

modell.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(lr=3e-4),
    metrics = ["accuracy"],
)
modell.fit(x_train, y_train, batch_size=100, epochs=5, verbose=2)
modell.evaluate(x_test, y_test, batch_size=32, verbose=2)


Train on 50000 samples
Epoch 1/5
50000/50000 - 84s - loss: 1.7034 - accuracy: 0.3804
Epoch 2/5
50000/50000 - 41s - loss: 1.3812 - accuracy: 0.5031
Epoch 3/5
50000/50000 - 41s - loss: 1.2663 - accuracy: 0.5515
Epoch 4/5
50000/50000 - 41s - loss: 1.1752 - accuracy: 0.5873
Epoch 5/5
50000/50000 - 41s - loss: 1.0988 - accuracy: 0.6142
10000/10000 - 38s - loss: 1.0657 - accuracy: 0.6260


[1.0656760743141174, 0.626]

### Model impovement by adding layers (BatchNorm, Dropout, Regularization L2)

In [18]:
# Now we will build model to improve accuracy :
def my_model():
    inputss = keras.Input(shape=(32,32,3))
    x= layers.Conv2D(32, 3, padding='valid')(inputss)
    # We saw that we did not use Activation coz we will use batchnorm layer and after that layer we will use Activation:
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3,)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputss =layers.Dense(10)(x)
    modelll = keras.Model(inputs= inputss, outputs=outputss)
    return modelll

modelll = my_model()

modelll.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(lr=3e-4),
    metrics = ["accuracy"],
)
modelll.fit(x_train, y_train, batch_size=100, epochs=5, verbose=2)
modelll.evaluate(x_test, y_test, batch_size=32, verbose=2)


Train on 50000 samples
Epoch 1/5
50000/50000 - 86s - loss: 1.4492 - accuracy: 0.4838
Epoch 2/5
50000/50000 - 94s - loss: 1.0793 - accuracy: 0.6215
Epoch 3/5
50000/50000 - 88s - loss: 0.9381 - accuracy: 0.6735
Epoch 4/5
50000/50000 - 83s - loss: 0.8353 - accuracy: 0.7112
Epoch 5/5
50000/50000 - 82s - loss: 0.7548 - accuracy: 0.7388
10000/10000 - 4s - loss: 1.0731 - accuracy: 0.6307


[1.0731352879524232, 0.6307]

In [21]:
# Now we will add Regularizer layer to overcome overfitting nd get better accuracy:
from tensorflow.keras import regularizers

def myy_model():
    inputss = keras.Input(shape=(32,32,3))
    x= layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputss)
    # We saw that we did not use Activation coz we will use batchnorm layer and after that layer we will use Activation:
    # Purpose of batchnormalization is to normalize data
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    # We will add Dropout here also :
    x = layers.Dropout(0.5)(x)
    outputss =layers.Dense(10)(x)
    modelll = keras.Model(inputs= inputss, outputs=outputss)
    return modelll

modelll = myy_model()

modelll.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(lr=3e-4),
    metrics = ["accuracy"],
)
modelll.fit(x_train, y_train, batch_size=100, epochs=5, verbose=2)
modelll.evaluate(x_test, y_test, batch_size=32, verbose=2)
# We have to increase epoch size coz we have also used Dropout :

Train on 50000 samples
Epoch 1/5
50000/50000 - 120s - loss: 3.4640 - accuracy: 0.3352
Epoch 2/5
50000/50000 - 115s - loss: 2.1887 - accuracy: 0.4847
Epoch 3/5
50000/50000 - 114s - loss: 1.7346 - accuracy: 0.5514
Epoch 4/5
50000/50000 - 117s - loss: 1.5262 - accuracy: 0.5865
Epoch 5/5
50000/50000 - 121s - loss: 1.4005 - accuracy: 0.6044
10000/10000 - 4s - loss: 1.3532 - accuracy: 0.6194


[1.3531849250793457, 0.6194]

## Model Subclassing with Keras:

In [11]:
xtrain = xtrain.reshape(-1, 28, 28, 1).astype("float32") / 255.0
xtest = xtest.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# We can make own Block instead of using layers multiple times...
# Like in previous example we used:    CNN..> ---- BATCH NORM...> ---- Relu....> 
# X10 
class CNNBlock(layers.Layer):
    def __init__(self, out_channals, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channals, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x= tf.nn.relu(x)
        return x
    
model = keras.Sequential(
    [ 
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10),     
    ]
)
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"],
)

model.fit(xtrain, ytrain, batch_size=44, epochs=3, verbose=2)
model.evaluate(xtest, ytest, batch_size=32, verbose=2)


Train on 60000 samples
Epoch 1/3
60000/60000 - 527s - loss: 2.3019 - accuracy: 0.1111
Epoch 2/3
60000/60000 - 532s - loss: 2.3013 - accuracy: 0.1124
Epoch 3/3
60000/60000 - 491s - loss: 2.3013 - accuracy: 0.1124
10000/10000 - 14s - loss: 2.3012 - accuracy: 0.1135


[2.3011566482543944, 0.1135]

In [1]:
'''

Something big custom model like Resnet using CNN Block : 

class CNNBlock(layers.Layer):
    def __init__(self, out_channals, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channals, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x= tf.nn.relu(x)

class ResBlock(layers.layer):
    def __init__(self, channels)
         super(ResBlock, self).__init__()
         self.ccn1 = CNNBlock(channals[0])
         self.ccn1 = CNNBlock(channals[1])
         self.ccn1 = CNNBlock(channals[2])
         self.pooling = layers.Maxpooling2D()
         self.identity_mapping = layers.conv2D(channals[1], 3, padding=[3] )
           
   def call(self, input_tensor, training= False):
         x = self.ccn1(input_tensor, training=training)
         x = self.ccn1(x, training=training)
         x = self.ccn1( x + self.identity_mapping(input_tensor), training=training,)
         return self.pooling(x)

class ResNet(keras.Model):
     def __init__ (self, num_classes):
        super(ResNet, self).__init__()
        self.Block1 = ResBlock([32,32,64])
        self.Block2 = ResBlock([128,128,256])
        self.Block3 = ResBlock([128,256,512])
        self.pool = layers.GlobalAveragePooling()
        self.classifier = layers.Dense(num_classes)
            
   def class(self, input_tensor, training = False)
          x = self.Block1(input_tensor, training=training)
          x = self.Block2(x, training=training)
          x = self.Block3(x, training=training)
          x = self.pool(x)
          return self.classifier(x)
         
'''

"\n\nSomething big custom model like Resnet using CNN Block : \n\nclass CNNBlock(layers.Layer):\n    def __init__(self, out_channals, kernel_size = 3):\n        super(CNNBlock, self).__init__()\n        self.conv = layers.Conv2D(out_channals, kernel_size, padding='same')\n        self.bn = layers.BatchNormalization()\n    def call(self, input_tensor, training=False):\n        x = self.conv(input_tensor)\n        x = self.bn(x, training=training)\n        x= tf.nn.relu(x)\n\nclass ResBlock(layers.layer):\n    def __init__(self, channels)\n         super(ResBlock, self).__init__()\n         self.ccn1 = CNNBlock(channals[0])\n         self.ccn1 = CNNBlock(channals[1])\n         self.ccn1 = CNNBlock(channals[2])\n         self.pooling = layers.Maxpooling2D()\n         self.identity_mapping = layers.conv2D(channals[1], 3, padding=[3] )\n           \n   def call(self, input_tensor, training= False):\n         x = self.ccn1(input_tensor, training=training)\n         x = self.ccn1(x, training=

In [2]:
# Custom Model :
"""
class my_model(keras.Model):
      def __init__(self, num_classes = 10):
         super(my_model, self).__init__()
         self.dense1 = layers.Dense(64)
         self.dense2 = layers.Dense(num_classes)
    
       def call(self, input_tensor):
          x = tf.nn.relu(self.dense1(input_tensor))
          return self.dense2(x)
       
"""

'\nclass my_model(keras.Model):\n      def __init__(self, num_classes = 10):\n         super(my_model, self).__init__()\n         self.dense1 = layers.Dense(64)\n         self.dense2 = layers.Dense(num_classes)\n    \n       def call(self, input_tensor):\n          x = tf.nn.relu(self.dense1(input_tensor))\n          return self.dense2(x)\n       \n'

In [3]:
# Custom Model with custom layers :
"""
class Dense(layer.layers):
      def __init__(self, units, input_dim):
          super(Dense, self).__init__()
          self.w = self.add_weight(name='w', shape=(input_dim, units), initializer='random_normal', trainable=True)
          self.b = self.add_weight(name='b', shape=(units, ), initializer='zeros', trainable=True)
          
      def call(self, inputs):
           return tf.matmul(inputs, self.w) + self.b

class my_model(keras.Model):
      def __init__(self, num_classes = 10):
         super(my_model, self).__init__()
         self.dense1 = Dense(64, 784)
         self.dense2 = Dense(10, 64)
    
       def call(self, input_tensor):
          x = tf.nn.relu(self.dense1(input_tensor))
          return self.dense2(x)
       
"""

"\nclass Dense(layer.layers):\n      def __init__(self, units, input_dim):\n          super(Dense, self).__init__()\n          self.w = self.add_weight(name='w', shape=(input_dim, units), initializer='random_normal', trainable=True)\n          self.b = self.add_weight(name='b', shape=(units, ), initializer='zeros', trainable=True)\n          \n      def call(self, inputs):\n           return tf.matmul(inputs, self.w) + self.b\n\nclass my_model(keras.Model):\n      def __init__(self, num_classes = 10):\n         super(my_model, self).__init__()\n         self.dense1 = Dense(64, 784)\n         self.dense2 = Dense(10, 64)\n    \n       def call(self, input_tensor):\n          x = tf.nn.relu(self.dense1(input_tensor))\n          return self.dense2(x)\n       \n"

In [5]:
# Custom Model regardless of input shapes :

"""
class Dense(layer.layers):
      def __init__(self, units):
          super(Dense, self).__init__()
          self.units = units
          
      def build(self, input_shape)
          self.w = self.add_weight(name='w', shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
          self.b = self.add_weight(name='b', shape=(self.units, ), initializer='zeros', trainable=True)
          
      def call(self, inputs):
           return tf.matmul(inputs, self.w) + self.b
           
"""
# Making own Relu Class:
"""
class MyRelu(layer.layers):
      def __init__(self):
         super(MyRelu, self).__init__()
      
      def call(self, x):
      return tf.math.maximum(x, 0)

class my_model(keras.Model):
      def __init__(self, num_classes = 10):
         super(my_model, self).__init__()
         self.dense1 = Dense(64)
         self.dense2 = Dense(num_classes)
    
       def call(self, input_tensor):
          x = MyRelu(self.dense1(input_tensor))
          return self.dense2(x)
"""

'\nclass MyRelu(layer.layers):\n      def __init__(self):\n         super(MyRelu, self).__init__()\n      \n      def call(self, x):\n      return tf.math.maximum(x, 0)\n\nclass my_model(keras.Model):\n      def __init__(self, num_classes = 10):\n         super(my_model, self).__init__()\n         self.dense1 = Dense(64)\n         self.dense2 = Dense(num_classes)\n    \n       def call(self, input_tensor):\n          x = MyRelu(self.dense1(input_tensor))\n          return self.dense2(x)\n'